In [ ]:
import pandas as pd

path_2018_22 = "시군구별 연령별 분만 통계 2018~2022.csv"
path_2023    = "건강보험심사평가원_시군구별 연령군별 분만 통계 2023.csv"
path_2024    = "건강보험심사평가원_시군구별 연령별 분만 통계 2024.csv"

df18_22 = pd.read_csv(path_2018_22, encoding="cp949")
df23    = pd.read_csv(path_2023,    encoding="cp949")
df24    = pd.read_csv(path_2024,    encoding="cp949")

df18_22 = df18_22.rename(columns={
    "년도": "연도"
})

df23 = df23.rename(columns={
    "진료년도": "연도",
    "연령군": "연령구분"
})

df24 = df24.rename(columns={
    "진료년도": "연도",
    "청구건수": "분만건수"
})

common_cols = ["연도", "시도", "시군구", "연령구분", "분만건수"]

df18_22 = df18_22[common_cols]
df23    = df23[common_cols]
df24    = df24[common_cols]

birth_all = pd.concat([df18_22, df23, df24], ignore_index=True)

birth_all.to_csv("시군구별_연령별_분만건수_2018_2024.csv",
                 index=False, encoding="utf-8-sig")

print(birth_all.head())
print(birth_all["연도"].unique())


     연도  시도  시군구    연령구분  분만건수
0  2018  서울  강남구  19세 이하     0
1  2018  서울  강남구  20~29세  1124
2  2018  서울  강남구  30~39세  8000
3  2018  서울  강남구  40~49세   628
4  2018  서울  강남구  50세 이상     0
[2018 2019 2020 2021 2022 2023 2024]


In [ ]:
import pandas as pd

path_2020 = "산후조리원 현황(2020).csv"
path_2023 = "보건복지부_전국 산후조리원 현황_20231231.csv"

df20 = pd.read_csv(path_2020, encoding="cp949")
df23 = pd.read_csv(path_2023, encoding="cp949")

df20["년도"] = 2020
df23["년도"] = 2023

postpartum = pd.concat([df20, df23], ignore_index=True)

postpartum = postpartum.drop(columns=["번호", "운영주체", "주소", "전화번호"], errors="ignore")

postpartum["일반실"] = pd.to_numeric(postpartum["일반실"], errors="coerce").fillna(0).astype(int)
postpartum["특실"]  = pd.to_numeric(postpartum["특실"],  errors="coerce").fillna(0).astype(int)

sido_order = postpartum["시도"].drop_duplicates().tolist()

postpartum["시도"] = pd.Categorical(
    postpartum["시도"],
    categories=sido_order,
    ordered=True
)

grouped = (
    postpartum
    .groupby(["년도", "시도", "시군구"], sort=False)
    .agg(
        산후조리원수=("산후조리원", "count"),
        일반실_병실수=("일반실", "sum"),
        특실_병실수=("특실", "sum")
    )
    .reset_index()
)

grouped = grouped.sort_values(["년도", "시도"])

grouped["총병실수"] = grouped["일반실_병실수"] + grouped["특실_병실수"]

grouped.to_csv("시군구별_산후조리원_통계_2020_2023.csv",
               index=False, encoding="utf-8-sig")



C:\Users\h4624\AppData\Local\Temp\ipykernel_20344\3039163109.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  postpartum


In [ ]:
import pandas as pd

paths = [
    "기관수현황_지역별의원표시과목별_기관수_2018년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2019년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2020년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2021년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2022년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2023년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2024년4분기.xlsx",
    "기관수현황_지역별의원표시과목별_기관수_2025년3분기.xlsx",
]

dfs = []

for path in paths:
    df = pd.read_excel(path, header=2)

    df = df[["기준분기", "시도", "시군구", "산부인과"]].copy()

    df = df[df["시군구"].astype(str).str.strip() != "계"]

    df["연도"] = df["기준분기"].astype(str).str[:4].astype(int)

    df["시도"] = df["시도"].replace("강원도", "강원특별자치도")

    df["시도"] = df["시도"].fillna("강원특별자치도")

    df = df.drop(columns=["기준분기"])

    dfs.append(df)

obgyn_all = pd.concat(dfs, ignore_index=True)

sido_order = dfs[0]["시도"].drop_duplicates().tolist()
obgyn_all["시도"] = pd.Categorical(
    obgyn_all["시도"],
    categories=sido_order,
    ordered=True
)

obgyn_all = obgyn_all.sort_values(["연도", "시도", "시군구"]).reset_index(drop=True)

obgyn_all.to_csv(
    "지역별_산부인과_기관수_2018_2025_강원특별자치도통일.csv",
    index=False,
    encoding="utf-8-sig"
)

print(obgyn_all.query("시도 == '강원특별자치도'").head())
print(obgyn_all["시도"].unique())


          시도  시군구  산부인과    연도
113  강원특별자치도  강릉시     6  2018
114  강원특별자치도  동해시     3  2018
115  강원특별자치도  삼척시     1  2018
116  강원특별자치도  속초시     4  2018
117  강원특별자치도  원주시     9  2018
['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', ..., '전라남도', '경상북도', '경상남도', '제주특별자치도', NaN]
Length: 18
Categories (17, object): ['서울특별시' < '부산광역시' < '대구광역시' < '인천광역시' ... '전라남도' < '경상북도' < '경상남도' < '제주특별자치도']


c:\Users\h4624\anaconda3\envs\sda2025\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\h4624\anaconda3\envs\sda2025\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\h4624\anaconda3\envs\sda2025\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\h4624\anaconda3\envs\sda2025\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\h4624\anaconda3\envs\sda2025\lib\site-packages\